# Advent of Code 2024

# Puzzle - part 1

**--- Day 4: Ceres Search ---**

"Looks like the Chief's not here. Next!" One of The Historians pulls out a device and pushes the only button on it. After a brief flash, you recognize the interior of the *Ceres monitoring station*!

As the search for the Chief continues, a small Elf who lives on the station tugs on your shirt; she'd like to know if you could help her with her **word search** (your puzzle input). She only has to find one word: XMAS.

This word search allows words to be horizontal, vertical, diagonal, written backwards, or even overlapping other words. It's a little unusual, though, as you don't merely need to find one instance of `XMAS` - you need to find **all of them**. Here are a few ways `XMAS` might appear, where irrelevant characters have been replaced with `.`:

```
..X...
.SAMX.
.A..A.
XMAS.S
.X....
```

The actual word search will be full of letters instead. For example:

```
MMMSXXMASM
MSAMXMSMSA
AMXSXMAAMM
MSAMASMSMX
XMASAMXAMM
XXAMMXXAMA
SMSMSASXSS
SAXAMASAAA
MAMMMXMMMM
MXMXAXMASX
```

In this word search, `XMAS` occurs a total of `18` times; here's the same word search again, but where letters not involved in any `XMAS` have been replaced with `.`:

```
....XXMAS.
.SAMXMS...
...S..A...
..A.A.MS.X
XMASAMX.MM
X.....XA.A
S.S.S.S.SS
.A.A.A.A.A
..M.M.M.MM
.X.X.XMASX
```

Take a look at the little Elf's word search.
**How many times does XMAS appear?**

## Input

In [127]:
# Load the input file

with open('input - Day 4.txt', 'r') as file:
    input = file.read()


print(input[:10])

XMMMSMMXMS


## Input Formatting

This is pretty much standard, we just split the one big input into lines.

In [128]:
from pprint import pprint

input_lines = input.split('\n')
pprint(input_lines[:10])

print(f"\nThere are {len(input_lines)} lines")

['XMMMSMMXMSXMXMAXSMMAXXXXSXXASXSASAMSXMAXAAMSMMMSXSAMXMMXMASMMMSXMXSAMXSASAMXXXXMMMMMMXMXSMXSMSMSMSXSSXSMSSMSAMMXXSAMXMXMXSMSMMASMXMAASMSMMMM',
 'MSMXAASXMSASAAAXSASMSAMXMMMXAAMMMASMMXSXMXSAMAAMXSMMASXSSXMAAAAAMAMAXASAXMMSMMMXSAAASMMAMSAMASAAASXMAMAAAAXMAMAASAMSAMXMAAMAMAASMMMSXMAAMAAM',
 'AAMMXXMXASAMXSSMSAMAXAMMSAXMMSMSSXMXSAXASXSASMXMAXASMMSAAASXMSSMMASMMMMMMAAAAXXAXMXMXAMAXMMSAMSMXMAXMXMMMSMSAMMXSAASASMMAXSASMMXAAAMMMSMSMXX',
 'SSSSXASMMMSMAXXAMXMXMXMASXSAAAAXMXXXMASAMMSAMASMMSAMAXXMSMMMMAMASAXAAXASAMXSSMMMSMSSSMMSSMXMAXMMSSMMXMMAMAASXSXMSMMSAMXSAXAMMASXMMMSAXAAAASX',
 'XAAMAMXAAAAMXMMMAAMAAAMMSXXMSSSMMSMAMAMAMAMXMAMAMAMMXMSXXMASMAXAMXSXMXMXXSAMAMAAAAAAXXAAAXXMMMMAAAAMMXSASMXMXMMMXMAMMMAMXSMAXAMAAMASXSMSXMAA',
 'MMMMAXSMMSSMSXAASXSASXSMMASAAAAMAAAMMMSXMXMAMXSAMSASAMXAXSMMSSMMXMAXMSMMAMXMAASXMSMMMMXSAMSMSMMMSXXMAASXXXAXAAAMAMSSMMXSASXSMAXMMMAXAXAMMSMA',
 'MXSSMMMSMMMAAMMMXAMXMXAAXAMMMSMMSMSXAXMAMAMASASASAASMAMSMSMAAMAXXMAXMAMMSSMSSXMMAXXXAAAXAASASMAAXASMMMSXMXMSMSMMASMXMAAMMSAXSAMS

In [129]:
# Check the last line, if it's empty then let's remove it
if input_lines[-1] == "":
    del input_lines[-1]

print(len(input_lines))

140


## Solution

So here is what I am thinking... how about we make this data into a grid format, where each character is a cell in a grid.
Then we iterate over that grid and everytime we "step" on an `X` we check in all directions for `M` then `A` and then `S`.

This will also give us a resonable complexity of **O(24n)** (6 directions \* 4 letters). Look... at least it's linear, okay?

In [130]:
# Making a class sounds fun, let's do that
# It's not the fastest way to do this but I want to do a class
# It will make our lives easier later, probably...

class Grid:
    def __init__(self, lines):

        self.grid = []

        # (0, 0) is top left corner
        self.current_x = 0 
        self.current_y = 0

        # These are for exploring
        self.delta_x = 0
        self.delta_y = 0 

        self.x_dimension= len(lines[0])
        self.y_dimension = len(lines)

        for line in lines:
            row = []
            for char in line:
                row.append(char)
            self.grid.append(row)

    # Cool, let's make it iterable
    def __iter__(self):
        for y, row in enumerate(self.grid):

            # Remember rows correspond to Ys
            self.current_y = y
            for x, col in enumerate(row):
                
                # and columns correspond to Xs (in the co-ordinate system)
                self.current_x = x
                yield col

    # Now we want to be able to explore in any direction
    # Notice that this is different that _iter_ it doesn't actually move
    # the reference point

    # Explore resetes the delta and brings us back to the reference co-ordinates
    def explore(self):
        self.delta_x = 0
        self.delta_y = 0 
        return self
    
    def up(self):
        self.delta_y -= 1
        return self

    def down(self):
        self.delta_y += 1
        return self

    def left(self):
        self.delta_x -= 1
        return self

    def right(self):
        self.delta_x += 1
        return self
    
    def move(self, direction:str):
        if direction == "up":
            return self.up()

        elif direction == "diag_up_right":
            return self.up().right()
        
        elif direction == "right":
            return self.right()        

        elif direction == "diag_down_right":
            return self.down().right()       

        elif direction == "down":
            return self.down()       

        elif direction == "diag_down_left":
            return self.down().left()       

        elif direction == "left":
            return self.left()
    
        elif direction == "diag_up_left":
            return self.up().left()
        
        else:
            raise NotImplemented
        
    def cords(self):
        effective_x = self.current_x + self.delta_x
        effective_y = self.current_y + self.delta_y

        # If we are outside the grid just return False
        if effective_x < 0 or effective_y < 0 or effective_x >= self.x_dimension or effective_y >= self.y_dimension:
            return False

        return effective_x, effective_y
    
    def value(self):
        cords = self.cords()

        # Need to handle the return False case
        if cords == False:
            return False

        effective_x, effective_y = cords

        return self.grid[effective_y][effective_x]


In [131]:
# X -> M -> A -> S

#     _________________
#    |__8__|__1__|__2__|
#    |__7__|__X__|__3__|
#    |__6__|__5__|__4__|

grid = Grid(input_lines)

count_matches = 0
# matches = []

for x in grid:
    directions = [
        "up",
        "diag_up_right",
        "right",
        "diag_down_right",
        "down",
        "diag_down_left",
        "left",
        "diag_up_left"
    ]

    if x == "X":
        next_letter = ['M', "A", "S"] 

        for direction in directions:
            # match = [[grid.cords(), "X"]]
            grid.explore()

            for expected_letter in next_letter:
                found_letter = grid.move(direction).value()

                if not found_letter or found_letter != expected_letter:
                    break
                
                if found_letter == expected_letter and expected_letter == "S":
                    # matches.append(match)
                    count_matches += 1


In [132]:
print(f"Found matches of XMAS: {count_matches}")

# pprint(matches)

Found matches of XMAS: 2297


# Puzzle - Part 2


**--- Part Two ---**

The Elf looks quizzically at you. Did you misunderstand the assignment?

Looking for the instructions, you flip over the word search to find that this isn't actually an **`XMAS`** puzzle; it's an **`X-MAS`** puzzle in which you're supposed to find two `MAS` in the shape of an `X`. One way to achieve that is like this:

```
M.S
.A.
M.S
```

Irrelevant characters have again been replaced with `.` in the above diagram. Within the `X`, each `MAS` can be written forwards or backwards.

Here's the same example from before, but this time all of the `X-MAS`es have been kept instead:

```
.M.S......
..A..MSMS.
.M.S.MAA..
..A.ASMSM.
.M.S.M....
..........
S.S.S.S.S.
.A.A.A.A..
M.M.M.M.M.
..........
```

In this example, `an X-MAS` appears **`9`** times.

Flip the word search from the instructions back over to the word search side and try again.
**How many times does an `X-MAS` appear?**

## Solution

This is just a variation of what we did above.
Instead of looking for `X`s we will look for `A`s and then check in four diagonal directions for letters.

We just have to be careful since on one cross-through diagonal (e.g from top right to bottom left) we can either match `M``S` or `S``M`.

In [133]:
# X -> M -> A -> S

#     _________________
#    |__1__|_____|__1__|
#    |_____|__A__|_____|
#    |__3__|_____|__3__|

grid = Grid(input_lines)

count_matches = 0
# matches = []

for a in grid:
    counted_diags = 0

    diag_directions = [
        [
            "diag_up_right",
            "diag_down_left"
        ],
        [
            "diag_up_left",
            "diag_down_right"
        ]

    ]

    if a == "A":
        possible_combinations = [['M', "S"], ["S", "M"]]

        for diag_1, diag_3 in diag_directions:
            letter_1 = grid.explore().move(diag_1).value()
            letter_3 = grid.explore().move(diag_3).value()

            for letter_set in possible_combinations:
                expected_l_1, expected_l_3 = letter_set

                if expected_l_1 == letter_1 and expected_l_3 == letter_3:
                    counted_diags += 1
                    break

    if counted_diags == 2:
        count_matches += 1

In [134]:
print(f"Found diagonal matches XMAS: {count_matches}")

Found diagonal matches XMAS: 1745
